## New Measure - Spend on BAD recommended specials as a proprotion of all  dermatological specials

## Rationale

Generally it is preferred that licensed medicines are use however in certain circumstances it may be necessary to use an unlicensed medicine, which gnerally will not have a substantial evidence base to justify widespread use. These medicines are generally made at the request of a prescription and are commonly referred to as _specials_.  

In primary care, unlike licensed medicines, there is a lack of price controls although introduction of Part VIIIB of the [NHS Drug Tariff](https://openprescribing.net/tariff/) in 2011 has exerted some degree of control. 

In dermatology _specials_ are commonly prescribed due to a perceived or otherwise lack of licensed products. Due to concerns over evidence, standards and costs the British Association of Dermatologists have produced [guidance](http://www.bad.org.uk/shared/get-file.ashx?itemtype=document&id=6248) to address these concerns and help to optimise quality of care.


## Proposed Measurse

### Title


### Description


### Why it matters

## Numerators and Demoninators

### SQL and results

In [2]:
#import libraries required for analysis
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ebmdatalab import bq
from ebmdatalab import charts
from ebmdatalab import maps

In [5]:
## Here we query our prescribing file to create a dataframe of all unlicnesed "creams"

sql='''SELECT
  month,
  bnf_name,
  presc.bnf_code,
  actual_cost,
  SUM(IF(form_route LIKE "%.cutaneous",
      items,
      0)) AS body,
  SUM(IF(form_route LIKE "%.scalp",
      items,
      0)) AS scalp
FROM
  ebmdatalab.hscic.prescribing_2019_01 AS presc
LEFT JOIN
  ebmdatalab.dmd.form_dose AS dmd
ON
  presc.bnf_code=dmd.bnf_code
  LEFT JOIN
ebmdatalab.dmd.product AS dmdprod ##need to check with Seb, Pete if this file is complete
ON
dmd.bnf_code=dmdprod.bnf_code
WHERE
(form_route LIKE "%.cutaneous"     ## this restricts to skin apllications
  OR form_route LIKE "%.scalp") AND ## this restricts to scalp applications
  ema = "false" ## this means it is not licnsed by the EMA
GROUP BY
  month,
  bnf_name,
  bnf_code,
  actual_cost'''

df_creams = bq.cached_read(sql, csv_path='df_creams.csv', use_cache=True)  # add `use_cache=False` to override
df_creams['month'] = df_creams['month'].astype('datetime64[ns]')
df_creams.head()

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


,month,bnf_name,bnf_code,actual_cost,body,scalp
0,2019-01-01,Betameth Diprop/Salic Acid_Oint 0.05/3%,1304000D0AABBBB,74.59850,14,0
1,2019-01-01,Clotrimazole_Spy 1% 40ml,1310020H0AAAEAE,8.77411,25,0
2,2019-01-01,Fludroxycortide_Tape 7.5cm x 20cm,1304000T0AACECE,253.76799,7,0
3,2019-01-01,Betameth Val/Clioquinol_Oint 0.1%/3%,1304000F0AACDCD,36.09878,560,0
4,2019-01-01,Sod Bicarb_(S),0101012B0AAAPAP,13.91396,1,0


In [9]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [11]:
df_creams.sum()

actual_cost   3189975.49
body          1967354.00
scalp          120726.00
dtype: float64

In [30]:
df_creams["total_items"] = df_creams.body + df_creams.scalp
df_creams.head()

,month,bnf_name,bnf_code,actual_cost,body,scalp,total_items
0,2019-01-01,Betameth Diprop/Salic Acid_Oint 0.05/3%,1304000D0AABBBB,74.60,14,0,14
1,2019-01-01,Clotrimazole_Spy 1% 40ml,1310020H0AAAEAE,8.77,25,0,25
2,2019-01-01,Fludroxycortide_Tape 7.5cm x 20cm,1304000T0AACECE,253.77,7,0,7
3,2019-01-01,Betameth Val/Clioquinol_Oint 0.1%/3%,1304000F0AACDCD,36.10,560,0,560
4,2019-01-01,Sod Bicarb_(S),0101012B0AAAPAP,13.91,1,0,1


In [32]:
df_creams.describe()

,actual_cost,body,scalp,total_items
count,43039.00,43039.00,43039.00,43039.00
mean,74.12,45.71,2.81,48.52
std,125.79,167.44,37.19,174.48
min,0.02,0.00,0.00,1.00
25%,21.13,8.00,0.00,9.00
50%,39.43,18.00,0.00,18.00
75%,74.63,36.00,0.00,38.00
max,3271.08,12480.00,1794.00,12480.00
